In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

import zipfile
import os
from pathlib import Path
import urllib.request


In [2]:
spark = SparkSession.builder \
    .appName("ner")\
    .master("local[1]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultS1ize", "2G") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.6.2")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [3]:
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

1. Download CoNLL2003 dataset
2. Save 3 files eng.train, eng.testa, eng.testa, into working dir ./

In [4]:
# Example how to download CoNLL 2003 Dataset
def download_conll2003_file(file):    
    if not Path(file).is_file():
        url = "https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/" + file
        urllib.request.urlretrieve(url, file)
        
download_conll2003_file("eng.train")
download_conll2003_file("eng.testa")
download_conll2003_file("eng.testb")

3 Download Glove word embeddings

In [5]:
file = "../../data/glove.6B.zip"
if not Path(file).is_file():
    url = "http://nlp.stanford.edu/data/glove.6B.zip"
    print("Start downoading Glove Word Embeddings. It will take some time, please wait...")
    urllib.request.urlretrieve(url, file)
    print("Downloading finished")
    
if not Path("../../data/glove.6B.100d.txt").is_file():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall("../../data/")
    zip_ref.close()


In [6]:
from pyspark.sql.types import *

class Annotation:
    def __init__(self, annotatorType, begin, end, result, metadata):
        self.annotatorType = annotatorType
        self.begin = begin
        self.end = end
        self.result = result
        self.metadata = metadata

        
annotation_schema = StructType([
    StructField("annotatorType", StringType()),
    StructField("begin", IntegerType(), False),
    StructField("end", IntegerType(), False),
    StructField("result", StringType()),
    StructField("metadata", MapType(StringType(), StringType()))
])
    


def readDataset(file, doc_column = "text", label_column = "label"):
    global spark
    
    result = []
    doc = ""
    labels = []

    with open(file) as f:
        for line in f:
            items = line.split(' ')
            word = items[0]
            if word == "-DOCSTART-":
                result.append((doc, labels))
                doc = ""
                labels = []
            elif len(items) <= 1:
                doc = doc + " \n"
            else:
                if len(doc) > 0:
                    doc = doc + " "

                begin = len(doc)
                doc = doc + word
                end = len(doc) - 1
                ner = items[3]
                labels.append(Annotation("named_entity", begin, end, ner, {}))

    if doc:
        result.append((doc, labels))
    
    global annotation_schema
    
    schema =  StructType([
      StructField(doc_column, StringType()),
      StructField(label_column, ArrayType(annotation_schema))
    ])
    
    
    return spark.createDataFrame(result, schema = schema)

In [7]:
import time

def get_pipeline():
    documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

    sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

    tokenizer = Tokenizer()\
      .setInputCols(["document"])\
      .setOutputCol("token")

    nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(10)\
      .setExternalDataset("file://" + os.getcwd() + "/eng.train")\
      .setEmbeddingsSource("file://" + os.getcwd() + "../../../data/glove.6B.100d.txt", 100, 2)\
      .setRandomSeed(0)\
      .setVerbose(2)

    converter = NerConverter()\
      .setInputCols(["document", "token", "ner"])\
      .setOutputCol("ner_span")
      
    pipeline = Pipeline(
        stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        nerTagger,
        converter
      ])
    
    return pipeline


def train_model(file):
    global spark
    
    print("Dataset Reading")
    
    start = time.time()
    dataset = readDataset(file)
    print("Done, {}\n".format(time.time() - start))

    print("Start fitting")
    pipeline = get_pipeline()

    return pipeline.fit(dataset)

In [8]:
from pyspark.sql.functions import col, udf, explode


def get_dataset_for_analysis(file, model):
    global spark
    
    print("Dataset Reading")
    
    start = time.time()
    dataset = readDataset(file)
    print("Done, {}\n".format(time.time() - start))
    
    predicted = model.transform(dataset)
    
    global annotation_schema
    
    zip_annotations = udf(
      lambda x, y: list(zip(x, y)),
      ArrayType(StructType([
          StructField("predicted", annotation_schema),
          StructField("label", annotation_schema)
      ]))
    )
    
    return predicted\
        .withColumn("result", zip_annotations("ner", "label"))\
        .select(explode("result").alias("result"))\
        .select(
            col("result.predicted").alias("predicted"), 
            col("result.label").alias("label")
        )
        
def printStat(label, correct, predicted, predictedCorrect):
    prec = predictedCorrect / predicted if predicted > 0 else 0
    rec = predictedCorrect / correct if correct > 0 else 0
    f1 = (2*prec*rec)/(prec + rec) if prec + rec > 0 else 0
    
    print("{}\t{}\t{}\t{}".format(label, prec, rec, f1))
        

def test_dataset(file, model, ignore_tokenize_misses=True):
    global spark
    
    started = time.time()

    df = readDataset(file)
    transformed = model.transform(df).select("label", "ner")

    labels = []
    predictedLabels = []

    for line in transformed.collect():
        label = line[0]
        ner = line[1]
        ner = {(a["begin"], a["end"]):a["result"] for a in ner}

        for a in label:
            key = (a["begin"], a["end"])

            label = a["result"].strip()
            predictedLabel = ner.get(key, "O").strip()
            
            if key not in ner and ignore_tokenize_misses:
                continue
                
            labels.append(label)
            predictedLabels.append(predictedLabel)
        

    correct = {}
    predicted = {}
    predictedCorrect = {}


    print(len(labels))

    for (lPredicted, lCorrect) in zip(predictedLabels, labels):
        correct[lCorrect] = correct.get(lCorrect, 0) + 1
        predicted[lPredicted] = predicted.get(lPredicted, 0) + 1

        if lCorrect == lPredicted:
            predictedCorrect[lPredicted] = predictedCorrect.get(lPredicted, 0) + 1

    correct = { key: correct[key] for key in correct.keys() if key != 'O'}
    predicted = { key: predicted[key] for key in predicted.keys() if key != 'O'}
    predictedCorrect = { key: predictedCorrect[key] for key in predictedCorrect.keys() if key != 'O'}

    tags = set(list(correct.keys()) + list(predicted.keys()))

    print("label\tprec\trec\tf1")
    totalCorrect = sum(correct.values())
    totalPredicted = sum(predicted.values())
    totalPredictedCorrect = sum(predictedCorrect.values())

    printStat("Total", totalCorrect, totalPredicted, totalPredictedCorrect)

    for label in tags:
        printStat(label, correct.get(label, 0), predicted.get(label, 0), predictedCorrect.get(label, 0))


In [9]:
import os.path

folder = '.'
train_file = os.path.join(folder, "eng.train")
test_file_a = os.path.join(folder, "eng.testa")
test_file_b = os.path.join(folder, "eng.testb")

In [10]:
model = train_model(train_file)

Dataset Reading
Done, 4.797255039215088

Start fitting


In [11]:
print("\nQuality on training data")
test_dataset(train_file, model)

print("\n\nQuality on validation data")
test_dataset(test_file_a, model)

print("\n\nQuality on test data")
test_dataset(test_file_b, model)


Quality on training data
5514
label	prec	rec	f1
Total	0.369140625	0.35526315789473684	0.3620689655172414
I-LOC	0.38524590163934425	0.33098591549295775	0.356060606060606
I-MISC	0.11904761904761904	0.08196721311475409	0.09708737864077668
I-ORG	0.3958333333333333	0.3931034482758621	0.39446366782006914
I-PER	0.39215686274509803	0.4371584699453552	0.41343669250646
B-MISC	0	0.0	0


Quality on validation data
1406
label	prec	rec	f1
Total	0.3357664233576642	0.3333333333333333	0.33454545454545453
I-PER	0.41818181818181815	0.35384615384615387	0.3833333333333333
I-MISC	0.0	0.0	0
I-LOC	0.3333333333333333	0.3870967741935484	0.3582089552238806
I-ORG	0.3333333333333333	0.39285714285714285	0.360655737704918


Quality on test data
1383
label	prec	rec	f1
Total	0.31386861313868614	0.33858267716535434	0.32575757575757575
I-LOC	0.36363636363636365	0.4444444444444444	0.39999999999999997
I-MISC	0.05555555555555555	0.1	0.07142857142857142
I-PER	0.37777777777777777	0.3953488372093023	0.38636363636363635
I-ORG

In [12]:
df = get_dataset_for_analysis(test_file_a, model)
df.show()

Dataset Reading
Done, 0.8117330074310303

+--------------------+--------------------+
|           predicted|               label|
+--------------------+--------------------+
|[named_entity, 0,...|[named_entity, 3,...|
|[named_entity, 8,...|[named_entity, 11...|
|[named_entity, 10...|[named_entity, 13...|
|[named_entity, 25...|[named_entity, 28...|
|[named_entity, 30...|[named_entity, 33...|
|[named_entity, 35...|[named_entity, 38...|
|[named_entity, 38...|[named_entity, 41...|
|[named_entity, 42...|[named_entity, 45...|
|[named_entity, 48...|[named_entity, 51...|
|[named_entity, 56...|[named_entity, 59...|
|[named_entity, 64...|[named_entity, 67...|
|[named_entity, 66...|[named_entity, 71...|
|[named_entity, 73...|[named_entity, 78...|
|[named_entity, 84...|[named_entity, 91...|
|[named_entity, 89...|[named_entity, 96...|
|[named_entity, 96...|[named_entity, 10...|
|[named_entity, 10...|[named_entity, 11...|
|[named_entity, 11...|[named_entity, 12...|
|[named_entity, 12...|[named_entit

In [13]:
get_pipeline().write().overwrite().save("./crf_pipeline")
model.write().overwrite().save("./crf_model")

In [14]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./crf_pipeline")
sameModel = PipelineModel.read().load("./crf_model")

In [15]:
print("\nQuality on training data")
test_dataset(train_file, sameModel)

print("\n\nQuality on validation data")
test_dataset(test_file_a, sameModel)

print("\n\nQuality on test data")
test_dataset(test_file_b, sameModel)


Quality on training data
5514
label	prec	rec	f1
Total	0.369140625	0.35526315789473684	0.3620689655172414
I-LOC	0.38524590163934425	0.33098591549295775	0.356060606060606
I-MISC	0.11904761904761904	0.08196721311475409	0.09708737864077668
I-ORG	0.3958333333333333	0.3931034482758621	0.39446366782006914
I-PER	0.39215686274509803	0.4371584699453552	0.41343669250646
B-MISC	0	0.0	0


Quality on validation data
1406
label	prec	rec	f1
Total	0.3357664233576642	0.3333333333333333	0.33454545454545453
I-PER	0.41818181818181815	0.35384615384615387	0.3833333333333333
I-MISC	0.0	0.0	0
I-LOC	0.3333333333333333	0.3870967741935484	0.3582089552238806
I-ORG	0.3333333333333333	0.39285714285714285	0.360655737704918


Quality on test data
1383
label	prec	rec	f1
Total	0.31386861313868614	0.33858267716535434	0.32575757575757575
I-LOC	0.36363636363636365	0.4444444444444444	0.39999999999999997
I-MISC	0.05555555555555555	0.1	0.07142857142857142
I-PER	0.37777777777777777	0.3953488372093023	0.38636363636363635
I-ORG